In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, GlobalMaxPool1D, Conv1D
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

## Dataset
dataset = [
    "မြန်မာ ပြည် ရှေ့ရေး အကြောင်း ဆွေးနွေးသွားတာ အရမ်း ကောင်းတယ်။", 
    "အရမ်း မှန်တဲ့ စကားတွေပါ။ ကျေးဇူး တင်ပါတယ်။", 
    "တန်ဖိုး ရှိတဲ့ စကားတွေပါ။ လေးစားပါတယ်။", 
    "သုံးသပ်သွားတာ ကောင်းလိုက်တာ။ အရမ်း ဗဟုသုတ ရတယ်။", 
    "ကျေးဇူးပါ ဆရာ သုံးသပ်ချက် တွေကို ကျေနပ် အားရပါတယ်။", 

    "မင်းဆိုး မင်းညစ် အယုတ်တမာ ကျဆုံးမှာပါ။", 
    "ကလိမ်ကကျစ် နဲ့ စကား ကို ဝေ့ဝိုက်ပြီး ပြောနေတယ်။", 
    "မိုက်ရိုက်လိုက်တဲ့ လုပ်ရပ်ဗျာ။ အတော် ဆိုးတာပဲ။", 
    "ကျက်သရေ တုံးပါကွာ။ အရည်အချင်းကို မရှိဘူး။", 
    "မင်းကို မြင်ရတာ စိတ် ကသိကအောက် ဖြစ်တယ်။", 

    "ပြည်ထောင်စု သမ္မတ မြန်မာ နိုင်ငံတော် ထာဝရ တည်တံ့နိုင်ပါစေ။", 
    "ပြည်ထောင်စု ဖွား တိုင်းရင်းသားတွေ စုစည်းနေရင် အရမ်း ကောင်းမှာပဲ။", 
    "စစ်ပွဲတွေ ကို မုန်းလိုက်တာ ဗျာ။", 
    "စာသင်ကျောင်း တံခါးများ အမြဲတမ်း ဖွင့်ထားနိုင်ပါစေ။", 
    "ရခိုင် မုန့်တီ၊ ရှမ်း ခေါက်ဆွဲ နဲ့ ကရင် ကုတိယို ကို အရမ်း ကြိုက်တယ်။" 
]
targets = ["Positive", "Positive", "Positive", "Positive", "Positive", "Negative", "Negative", "Negative", "Negative", "Negative", "Neutral", "Neutral", "Neutral", "Neutral", "Neutral"]

## Parameters
VOCAB_SIZE = 100
MAX_LENGTH = 15

In [7]:
# Convert documents to one-hot encoded representation
encoded_docs = [one_hot(doc, VOCAB_SIZE) for doc in dataset]
encoded_docs

[[95, 67, 28, 77, 19, 14, 16],
 [14, 31, 11, 34, 58],
 [87, 42, 11, 12],
 [9, 54, 14, 36, 55],
 [79, 87, 44, 81, 79, 55],
 [80, 61, 92, 39],
 [83, 33, 75, 86, 93, 72],
 [83, 59, 83, 74],
 [41, 16, 93, 10],
 [10, 87, 25, 70, 86],
 [89, 16, 95, 41, 93, 81],
 [89, 19, 71, 40, 14, 23],
 [97, 86, 26, 37],
 [67, 84, 37, 45],
 [18, 5, 27, 81, 33, 54, 21, 86, 14, 33]]

In [8]:
# Pad sequences to ensure consistent input size
pad_sentences = pad_sequences(encoded_docs, maxlen=MAX_LENGTH, padding='post')
print(f"Padding : {pad_sentences}")

Padding : [[95 67 28 77 19 14 16  0  0  0  0  0  0  0  0]
 [14 31 11 34 58  0  0  0  0  0  0  0  0  0  0]
 [87 42 11 12  0  0  0  0  0  0  0  0  0  0  0]
 [ 9 54 14 36 55  0  0  0  0  0  0  0  0  0  0]
 [79 87 44 81 79 55  0  0  0  0  0  0  0  0  0]
 [80 61 92 39  0  0  0  0  0  0  0  0  0  0  0]
 [83 33 75 86 93 72  0  0  0  0  0  0  0  0  0]
 [83 59 83 74  0  0  0  0  0  0  0  0  0  0  0]
 [41 16 93 10  0  0  0  0  0  0  0  0  0  0  0]
 [10 87 25 70 86  0  0  0  0  0  0  0  0  0  0]
 [89 16 95 41 93 81  0  0  0  0  0  0  0  0  0]
 [89 19 71 40 14 23  0  0  0  0  0  0  0  0  0]
 [97 86 26 37  0  0  0  0  0  0  0  0  0  0  0]
 [67 84 37 45  0  0  0  0  0  0  0  0  0  0  0]
 [18  5 27 81 33 54 21 86 14 33  0  0  0  0  0]]


In [9]:
# Encoding the target labels
encoder = LabelEncoder()
encoded_lbls = encoder.fit_transform(targets)
print(f'Encoded labels : {encoded_lbls}')

Encoded labels : [2 2 2 2 2 0 0 0 0 0 1 1 1 1 1]


In [10]:
## Custom Embedding Class (Modified for Keras Model)
class CustomEmbedding(Layer):
    def __init__(self, input_dim, output_dim):
        super(CustomEmbedding, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

    def build(self, input_shape):
        # Initialize embedding matrix with random values
        self.embeddings = self.add_weight(
            name='embeddings',
            shape=(self.input_dim, self.output_dim),
            initializer='random_normal',
            trainable=True
        )

    def call(self, inputs):
        # Lookup embeddings for each index in the input sequence
        return tf.gather(self.embeddings, inputs)

In [11]:
## Create CNN Model with CustomEmbedding
model = Sequential()
model.add(CustomEmbedding(input_dim=VOCAB_SIZE, output_dim=10))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(3, activation='softmax'))

# Now, we call the model on a batch of data to ensure the model is built
model(np.zeros((1, MAX_LENGTH), dtype=np.int32))  # Pass a dummy batch with integer data
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 custom_embedding (CustomEmb  (1, 15, 10)              1000      
 edding)                                                         
                                                                 
 conv1d (Conv1D)             (1, 11, 128)              6528      
                                                                 
 global_max_pooling1d (Globa  (1, 128)                 0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (1, 3)                    387       
                                                                 
Total params: 7,915
Trainable params: 7,915
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
## Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Train Model
model.fit(pad_sentences, encoded_lbls, epochs=150, verbose=1)

Epoch 1/150
1/1 [==============================] - 0s 296ms/step - loss: 1.0995 - accuracy: 0.2667
Epoch 2/150
1/1 [==============================] - 0s 3ms/step - loss: 1.0925 - accuracy: 0.3333
Epoch 3/150
1/1 [==============================] - 0s 2ms/step - loss: 1.0858 - accuracy: 0.8000
Epoch 4/150
1/1 [==============================] - 0s 2ms/step - loss: 1.0796 - accuracy: 0.8667
Epoch 5/150
1/1 [==============================] - 0s 3ms/step - loss: 1.0735 - accuracy: 0.8667
Epoch 6/150
1/1 [==============================] - 0s 2ms/step - loss: 1.0676 - accuracy: 0.8667
Epoch 7/150
1/1 [==============================] - 0s 3ms/step - loss: 1.0617 - accuracy: 0.9333
Epoch 8/150
1/1 [==============================] - 0s 7ms/step - loss: 1.0558 - accuracy: 0.9333
Epoch 9/150
1/1 [==============================] - 0s 5ms/step - loss: 1.0500 - accuracy: 1.0000
Epoch 10/150
1/1 [==============================] - 0s 5ms/step - loss: 1.0440 - accuracy: 1.0000
Epoch 11/150
1/1 [=========

In [13]:
## Model Evaluation
loss, acc = model.evaluate(pad_sentences, encoded_lbls, verbose=0)

# Print the results
print(f'Accuracy : {acc}, Loss : {loss}')

Accuracy : 1.0, Loss : 0.008903194218873978


In [14]:
test = [
    "တန်ဖိုး ရှိတဲ့ စကားတွေပါ။ လေးစားပါတယ်။", 
    "မိုက်ရိုက်လိုက်တဲ့ လုပ်ရပ်ဗျာ။ အတော် ဆိုးတာပဲ။",
    "စာသင်ကျောင်း တံခါးများ အမြဲတမ်း ဖွင့်ထားနိုင်ပါစေ။"
]
encoded_test_docs = [one_hot(doc, VOCAB_SIZE) for doc in test]
pad_test_sentences = pad_sequences(encoded_test_docs, maxlen=MAX_LENGTH, padding='post')

## Predict using the trained model
predictions = model.predict(pad_test_sentences, verbose=0)
predicted_labels = np.argmax(predictions, axis=1)
predicted_classes = encoder.inverse_transform(predicted_labels)

## Print the predictions
for i, sentence in enumerate(test):
    print(f"'{sentence}' -> {predicted_classes[i]}")

'တန်ဖိုး ရှိတဲ့ စကားတွေပါ။ လေးစားပါတယ်။' -> Positive
'မိုက်ရိုက်လိုက်တဲ့ လုပ်ရပ်ဗျာ။ အတော် ဆိုးတာပဲ။' -> Negative
'စာသင်ကျောင်း တံခါးများ အမြဲတမ်း ဖွင့်ထားနိုင်ပါစေ။' -> Neutral
